<a href="https://colab.research.google.com/github/CoDeRgAnEsh/Colab/blob/master/OR_ASSIGNMENT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OR Assignment - 3 



[Assignment problem](https://en.wikipedia.org/wiki/Assignment_problem)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CoDeRgAnEsh/Colab/blob/master/OR_ASSIGNMENT_3.ipynb)




---



Created by [Ganesh Kumar T K](https://github.com/coderganesh), MSM17B034

Dept. of Smart Manufacturing,

[IIITDM Kancheepuram](https://iiitdm.ac.in)

---



Code base is built upon Python 3.7

Works on Python == 3.4 +

Libraries used:

- copy - to deep copy matrix and array rows and cols.
- numpy - handle larger array operations using  C wrapper.


In [0]:
#@title Implementation of Hungarian method. Run this to proceed (Double-click to view code)
import copy

# Zeros in the algorithm can be starred or primed, or neither
NEITHER = 0
STARRED = 1
PRIMED = 2

# Step 1
# For each row, find the min in that row, and subtract it from every cell in
# that row 
def subtractRowMinFromEachRow(grid):
    for r in range(len(grid)):
        min_ = min(grid[r])
        for c in range(len(grid[r])):
            grid[r][c] -= min_

# Step 2
# For each uncovered zero in the grid, add a star to it and then cover that row and column.
# But then uncover all the rows and cols
def starZeros(grid, grid_info, row_cover, col_cover):
    for r in range(len(grid)):
        for c in range(len(grid[r])):
            if grid[r][c] == 0 and row_cover[r] == False and col_cover[c] == False:
                grid_info[r][c] = STARRED
                row_cover[r] = col_cover[c] = True

    uncoverAll(row_cover, col_cover)

# Step 3
# Cover all the columns with starred zeros, and return the number of covered columns
def coverColsWithStarredZeros(grid, grid_info, col_cover):
    for r in range(len(grid)):
        for c in range(len(grid[r])):
            if grid_info[r][c] == STARRED:
                # cover the column
                col_cover[c] = True

    covered_cnt = 0
    for c in range(len(col_cover)):
        if col_cover[c]:
            covered_cnt += 1

    return covered_cnt

# Return the column of a starred zero in the specified row, or -1
def isStarInRow(row, grid, grid_info):
    for c in range(len(grid[row])):
        if grid_info[row][c] == STARRED:
            return c

    return -1

# Return the row of a starred zero in the specified column, or -1
def isStarInCol(col, grid, grid_info):
    for r in range(len(grid)):
        if grid_info[r][col] == STARRED:
            return r

    return -1

# Find an uncovered zero, and return it's row and column (-1 returned if one is not present)
def findZero(grid, grid_info, row_cover, col_cover):
    for r in range(len(grid)):
        for c in range(len(grid[r])):
            if grid[r][c] == 0 and row_cover[r] == False and col_cover[c] == False:
                return r, c

    return -1, -1

# Step 4
# Keep finding non covered zeros, prime them, and see if there is a starred zero in that row (that you found the non covered zero in).
# If there is no starred zero in that row, return the row and column you just visited, otherwise mark the row as covered, and the column
# you saw the starred zero in as covered
def primeZeros(grid, grid_info, row_cover, col_cover):
    while True:
        r, c = findZero(grid, grid_info, row_cover, col_cover)
        if r == -1:
            return False

        grid_info[r][c] = PRIMED
        col_of_star = isStarInRow(r, grid, grid_info)
        if col_of_star == -1:
            return {'row' : r, 'col' : c}

        row_cover[r] = True
        col_cover[col_of_star] = False

# Find the smallest number in the grid that is not covered
def findSmallestUncoveredValue(grid, grid_info, row_cover, col_cover):
    min_ = None
    for r in range(len(grid)):
        for c in range(len(grid[r])):
            if row_cover[r] == 0 and col_cover[c] == 0 and (min_ == None or grid[r][c] < min_):
                min_ = grid[r][c]

    return min_

# step 6
# For each number in the grid, if it's row is covered, add the min to its value. 
# If the column is not covered, then subtract the min for it
def applySmallestValue(grid, grid_info, row_cover, col_cover, min_):
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if row_cover[r]:
                grid[r][c] += min_
            if not col_cover[c]:
                grid[r][c] -= min_

# Toggle the star in every point in the path
def augmentPath(grid, grid_info, path):
    for pair in path:
        r = pair['row']
        c = pair['col']
        if grid_info[r][c] == STARRED:
            grid_info[r][c] = NEITHER
        else:
            grid_info[r][c] = STARRED

# Remove the prime from every primed item in the path
def unprimeAll(grid_info):
    for r in range(len(grid_info)):
        for c in range(len(grid_info[r])):
            if grid_info[r][c] == PRIMED:
                grid_info[r][c] = NEITHER

# Uncover all the rows and cols
def uncoverAll(row_cover, col_cover):
    for i in range(len(row_cover)):
        row_cover[i] = False
    for i in range(len(col_cover)):
        col_cover[i] = False

# Return the column of a primed zero in the specified row, or -1
def isPrimeInRow(row, grid, grid_info):
    for c in range(len(grid[row])):
        if grid_info[row][c] == PRIMED:
            return c

    return -1

# Step 5
# Construct a series of alternating primed and starred zeros as follows.  Let Z0 represent the uncovered primed zero found in Step 4.  
# Let Z1 denote the starred zero in the column of Z0 (if any). Let Z2 denote the primed zero in the row of Z1 (there will always be one).  
# Continue until the series terminates at a primed zero that has no starred zero in its column.  
# Unstar each starred zero of the series, star each primed zero of the series, erase all primes and uncover every line in the matrix.  
# Return to Step 3. 
def pathStuff(grid, grid_info, row_cover, col_cover, pair):
    path = []
    path.append(pair)
    done = False
    while not done:
        row = isStarInCol(path[-1]['col'], grid, grid_info)
        if row > -1:
            path.append({"row" : row, "col" : path[-1]['col']})
        else:
            done = True
        if not done:
            col = isPrimeInRow(path[-1]['row'], grid, grid_info)
            path.append({"row" : path[-1]['row'], "col" : col})

    augmentPath(grid, grid_info, path)
    unprimeAll(grid_info)
    uncoverAll(row_cover, col_cover)

# For every element in the grid, set grid[r][c] = max_value_of_grid - grid[r][c]
# This is the trick to turn hungarian minimization into maximization
def normalize(grid):
    max_ = grid[0][0]
    for row in grid:
        max_of_row = max(row)
        if max_of_row > max_:
            max_ = max_of_row

    for r in range(len(grid)):
        for c in range(len(grid[r])):
            grid[r][c] = max_ - grid[r][c]
    return max_

# Transpose a grid
def transpose(grid):
    new_grid = []
    for c in range(len(grid[0])):
        new_grid.append([])
        for r in range(len(grid)):
            new_grid[c].append(grid[r][c])

    return new_grid

# Returns a matrix where an item is set to 1 if that row, col is part of the extrema
def assign(grid):
    # all rows and cols start off uncovered
    row_cover = [False]*len(grid) 
    col_cover = [False]*len(grid[0])
    # all numbers are non starred, and non primed
    # 1 means it's starred, 2 means it's primed
    grid_info = [[NEITHER]*len(grid[0]) for i in grid]
    # exit when the number of covered zeros equals this
    exit_condition = min(len(grid), len(grid[0])) 
            
    # Step 1
    subtractRowMinFromEachRow(grid)
    # Step 2
    starZeros(grid, grid_info, row_cover, col_cover)
    cover = True
    while True:
        if cover:
            # step 3
            covered_cnt = coverColsWithStarredZeros(grid, grid_info, col_cover)
            if covered_cnt >= exit_condition:
                break
    
        # step 4
        pair = primeZeros(grid, grid_info, row_cover, col_cover)
        if pair:
            # step 5
            pathStuff(grid, grid_info, row_cover, col_cover, pair)
            # go to step 3
            cover = True
        else:
            # step 6
            n = findSmallestUncoveredValue(grid, grid_info, row_cover, col_cover)
            applySmallestValue(grid, grid_info, row_cover, col_cover, n)
            # goto step 4
            cover = False

    return grid_info

# Transpose the matrix (if it has more rows than cols)
# And, if the method is maximization, perform the subtraction rule (the normalization function)
def prepare(grid, method="max"):
    rows = len(grid)
    cols = len(grid[0])
    # The algo requires that the number of cols >= the number of rows
    if rows > cols:
        grid = transpose(grid)
    working_copy = copy.deepcopy(grid)

    # Make the adjustments for maximization 
    if method == "max":
        normalize(working_copy)

    return grid, working_copy

# sum up the starred items in the grid
def sum(grid, grid_info):
    sum = 0
    for r in range(len(grid_info)):
        for c in range(len(grid_info[r])):
            if grid_info[r][c] == STARRED:
                sum += grid[r][c]

    return sum

# Return a list of the tuples (row, column,) that correspond to the extrema points
def listPoints(grid_info):
    rows = len(grid_info)
    cols = len(grid_info[0])
    # If we transposed the grid earlier, transpose it again to get the proper coordinates
    if rows != cols:
        grid_info = transpose(grid_info)

    points = []
    for r in range(len(grid_info)):
        for c in range(len(grid_info[r])):
            if grid_info[r][c] == STARRED:
                points.append((r, c))

    return points

# Prepares the matrix for assignment, performs the assignment, and returns the extrema and list of points
# where the extrema occur
def solve(grid, method="min"):
    grid, working_copy = prepare(grid, method)
    grid_info = assign(working_copy)
    return sum(grid, grid_info), listPoints(grid_info)

# convience methods
def minimize(grid):
    return solve(grid, "min") 
def maximize(grid):
    return solve(grid, "max")

#1. 

Batsman replacement problem

Since for replacement I'm adding dummy position


|  Batsman 	|   I	|   II	|   III	|  IV 	|   V	|   Dummy	|
|:-:	|:-:	|:-:	|:-:	|:-:	|:-:	|:-:	|
|  P 	|  40 	|  40 	| 35  	|  25 	| 50  	|   0	|
|   Q	|   42	|   30	|   16	|   25	|   27	|   0	|
|   R	|  50 	|   	48|40   	|   	60|50   	|   0	|
|   S	|   20	|   19	|  20 	|   18	|  25 	|   0	|
|   T	|   	58|  60 	|   59	|  55 	|   53	|   0	|
|   U	|   	45|52   	|   	38|50   	|   	49|   0	|

In [0]:
import numpy as np
position = ['I', 'II', 'III', 'IV', 'V', 'Dummy']
players = ['P', 'Q', 'R', 'S', 'T', 'U']

grid =np.array([[40, 40, 35, 25, 50, 0], [42, 30, 16, 25, 27, 0], [50, 48, 40, 60, 50, 0], [20, 19, 20, 18, 25, 0], [58, 60, 59, 55, 53, 0],[45, 52, 38, 50, 49, 0 ]])
#grid =np.array([[4, 6, 10, 5, 6], [7, 4, np.inf, 5, 4], [np.inf, 6, 9, 6, 2], [9, 3, 7, 2, 3], [0, 0, 0, 0, 0]])

total, coordinates = maximize(grid)
print ("The optimal assignment of the amount of runs to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", position[col], "to", players[row])

The optimal assignment of the amount of runs to complete all position is 263
Assign V to P
Assign I to Q
Assign IV to R
Assign Dummy to S
Assign III to T
Assign II to U


Therefore, U replaces S !

#2. 
As the matix is non-square, adding dummy machine with Zero cost. 

> As M2 cannot be place at c and M3 cannot be placed at A, Assigning them infinite Cost

|   Machine	|   A	|  B 	|   C	|  D 	|  E 	|
|:-:	|:-:	|:-:	|:-:	|:-:	|:-:	|
|   M1	|4   	|   6	|10   	|   5	|  6 	|
|   M2	| 7  	|   4	|  INF 	|   5	|   4	|
|   M3	|  INF 	| 6  	|   9	|  6 	|   2	|
|   M4	|   	9| 3  	|   7	| 2  	|   3	|
|  Dummy|   	0|0   	| 0  	| 0  	| 0  	|

In [0]:
import numpy as np
machine = ['I', 'II', 'III', 'IV', 'Dummy',]
workers = ['A', 'B', 'C', 'D', 'E', 'F']

#grid =np.array([[40, 40, 35, 25, 50, 0], [42, 30, 16, 25, 27, 0], [50, 48, 40, 60, 50, 0], [20, 19, 20, 18, 25, 0], [58, 60, 59, 55, 53, 0],[45, 52, 38, 50, 49, 0 ]])
grid =np.array([[4, 6, 10, 5, 6], [7, 4, np.inf, 5, 4], [np.inf, 6, 9, 6, 2], [9, 3, 7, 2, 3], [0, 0, 0, 0, 0]])

total, coordinates = minimize(grid)
print ("The optimal assignment of the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", machine[col], "to", workers[row])

The optimal assignment of the amount of cost to complete all position is 12.0
Assign I to A
Assign II to B
Assign Dummy to C
Assign IV to D
Assign III to E


#3. 

|  * 	|   M1	| M2  	| M3  	| M4  	|
|:-:	|:-:	|:-:	|:-:	|:-:	|
|   J1	|   5	|  7 	|  11 	|  6 	|
|   	J2|  8 	|  5 	|   9	|   6	|
|   J3	| 4	|   7	|   10	|   7	|
|   	J4|   10	|  4 	|  8 	|   3	|

In [0]:
import numpy as np
machine = ['M1', 'M2', 'M3', 'M4']
jobs = ['J1', 'J2', 'J3', 'J4']

grid =np.array([[5, 7, 11, 6], [8, 5, 9, 6], [4, 7, 10, 7], [10, 4, 8, 3]])

total, coordinates = minimize(grid)
print ("The optimal assignment of the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", machine[col], "to", jobs[row])

The optimal assignment of the amount of cost to complete all position is 23
Assign M2 to J1
Assign M3 to J2
Assign M1 to J3
Assign M4 to J4


#4.

|  * 	|   M1	| M2  	| M3  	| M4  	|
|:-:	|:-:	|:-:	|:-:	|:-:	|
|   J1	|   15	|  13 	|  14 	|  17 	|
|   	J2|  11 	|  12 	|   15	|   13	|
|   J3	| 13	|   12	|   10	|   11	|
|   	J4|   15	|  17 	|  14 	|   16	|

In [0]:
import numpy as np
machine = ['M1', 'M2', 'M3', 'M4']
jobs = ['J1', 'J2', 'J3', 'J4']

grid =np.array([[15, 13, 14, 17], [11, 12, 15, 13], [13, 12, 10, 11], [15, 17, 14, 16]])

total, coordinates = minimize(grid)
print ("The optimal assignment of *minimized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", machine[col], "to", jobs[row])

The optimal assignment of *minimized* the amount of cost to complete all position is 49
Assign M2 to J1
Assign M1 to J2
Assign M4 to J3
Assign M3 to J4


#5. 

In [0]:
import numpy as np
machine = ['M1', 'M2', 'M3', 'M4']
jobs = ['J1', 'J2', 'J3', 'J4']

grid =np.array([[2, 3, 4, 5], [4, 5, 6, 7], [7, 8, 9, 8], [3, 5, 8, 4]])

total, coordinates = minimize(grid)
print ("The optimal assignment of *minimized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", machine[col], "to", jobs[row])

The optimal assignment of *minimized* the amount of cost to complete all position is 20
Assign M3 to J1
Assign M2 to J2
Assign M4 to J3
Assign M1 to J4


#6. 

In [0]:
import numpy as np
machine = ['M1', 'M2', 'M3', 'M4','M5']
jobs = ['J1', 'J2', 'J3', 'J4','J5']

grid =np.array([[3,2,7,4,8], [5,4,3,8,5], [3,7,9,1,2], [4,2,6,5,7], [2,8,4,6,6]])

total, coordinates = minimize(grid)
print ("The optimal assignment of *minimized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", machine[col], "to", jobs[row])

The optimal assignment of *minimized* the amount of cost to complete all position is 13
Assign M4 to J1
Assign M3 to J2
Assign M5 to J3
Assign M2 to J4
Assign M1 to J5


#7.
# **(MAXIMIZE)**

In [0]:
import numpy as np
products = ['A', 'B', 'C', 'D']
operators = ['P', 'Q', 'R', 'S']

grid =np.array([[240,300,225,300], [360,450,250,300], [144,180,150,200], [240,300,225,180]])

total, coordinates = maximize(grid)
print ("The optimal assignment of *maximized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", products[col], "to", operators[row])

The optimal assignment of *maximized* the amount of cost to complete all position is 1140
Assign D to P
Assign B to Q
Assign C to R
Assign A to S


#8.

In [0]:
import numpy as np
products = ['A', 'B', 'C', 'D']
operators = ['P', 'Q', 'R', 'S']

grid =np.array([[2, 3, 4, 5], [4, 5, 6, 7], [7, 8, 9, 8], [3, 5, 8, 4]])

total, coordinates = minimize(grid)
print ("The optimal assignment of *minimized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", products[col], "to", operators[row])

total, coordinates = maximize(grid)
print ("The optimal assignment of *maximized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", products[col], "to", operators[row])

The optimal assignment of *minimized* the amount of cost to complete all position is 20
Assign C to P
Assign B to Q
Assign D to R
Assign A to S
The optimal assignment of *maximized* the amount of cost to complete all position is 25
Assign D to P
Assign A to Q
Assign B to R
Assign C to S


#9.

In [0]:
import numpy as np
machine = ['M1', 'M2', 'M3', 'M4','M5']
jobs = ['J1', 'J2', 'J3', 'J4','J5']

grid =np.array([[10,5,13,15,16], [3,9,18,3,6], [10,7,2,2,2], [5,11,9,7,12], [7,9,10,4,12]])
print ('The given matrix is:\n', grid)
total, coordinates = minimize(grid)
print ("The optimal assignment of *minimized* the amount of cost to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", machine[col], "to", jobs[row])

The given matrix is:
 [[10  5 13 15 16]
 [ 3  9 18  3  6]
 [10  7  2  2  2]
 [ 5 11  9  7 12]
 [ 7  9 10  4 12]]
The optimal assignment of *minimized* the amount of cost to complete all position is 22
Assign M2 to J1
Assign M5 to J2
Assign M3 to J3
Assign M1 to J4
Assign M4 to J5


#10.

In [0]:
import numpy as np
position = ['I', 'II', 'III', 'IV', 'V']
players = ['P', 'Q', 'R', 'S', 'T']

grid =np.array([[10,3,3,2,8], [9,7,8,2,7], [7,5,6,2,4], [3,5,8,2,4], [9,10,9,6,10]])

total, coordinates = maximize(grid)
print ("The optimal assignment of the amount of runs to complete all position is", total)
for (row, col) in coordinates:
    print ("Assign", position[col], "to", players[row])

The optimal assignment of the amount of runs to complete all position is 37
Assign V to P
Assign I to Q
Assign IV to R
Assign III to S
Assign II to T
